In [1]:
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setAppName("recommender")
sc = SparkContext(conf=conf)

In [3]:
def load_data(path, header, token_fun):
    return sc.textFile(path).filter(lambda x: x!=header).map(lambda x: x.split(",")).map(token_fun).cache()

links = load_data('ml-latest-small/links.csv', 
                  'movieId,imdbId,tmdbId', 
                  lambda tokens: (int(tokens[0]), int(tokens[1])))

movies = load_data('ml-latest-small/movies.csv', 
                   'movieId,title,genres', 
                   lambda tokens: (int(tokens[0]),tokens[1]))

ratings = load_data('ml-latest-small/ratings.csv', 
                    'userId,movieId,rating,timestamp', 
                    lambda tokens: (int(tokens[0]), int(tokens[1]), float(tokens[2])))

tags = load_data('ml-latest-small/tags.csv', 
                    'userId,movieId,tag,timestamp', 
                    lambda tokens: (int(tokens[0]), int(tokens[1]), tokens[2]))

In [4]:
movies.take(5)

[(1, 'Toy Story (1995)'),
 (2, 'Jumanji (1995)'),
 (3, 'Grumpier Old Men (1995)'),
 (4, 'Waiting to Exhale (1995)'),
 (5, 'Father of the Bride Part II (1995)')]

In [5]:
def split_sets(ratings, proportions):
    split = ratings.randomSplit(proportions)
    return {'training': split[0], 'validation': split[1], 'test': split[2]}
    
sets = split_sets(ratings, [0.63212056, 0.1839397, 0.1839397])

In [6]:
from pyspark.mllib.recommendation import ALS
rank = 10
iterations = 10
seed = 42
model = ALS.train(sets['training'], rank, seed=seed, iterations=iterations)

Py4JJavaError: An error occurred while calling z:org.apache.spark.mllib.api.python.SerDe.pythonToJava.
: java.lang.IllegalArgumentException
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.xbean.asm5.ClassReader.<init>(Unknown Source)
	at org.apache.spark.util.ClosureCleaner$.getClassReader(ClosureCleaner.scala:46)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:449)
	at org.apache.spark.util.FieldAccessFinder$$anon$3$$anonfun$visitMethodInsn$2.apply(ClosureCleaner.scala:432)
	at scala.collection.TraversableLike$WithFilter$$anonfun$foreach$1.apply(TraversableLike.scala:733)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashMap$$anon$1$$anonfun$foreach$2.apply(HashMap.scala:103)
	at scala.collection.mutable.HashTable$class.foreachEntry(HashTable.scala:230)
	at scala.collec

In [7]:
validation = sets['validation'].map(lambda x: (x[0], x[1]))
validation.take(5)

[(1, 3), (1, 6), (1, 50), (1, 235), (1, 316)]

In [8]:
predictions = model.predictAll(validation)
predictions.take(10)

NameError: name 'model' is not defined